<a href="https://colab.research.google.com/github/aaroncanillas/llm-playground/blob/main/llm_huggingface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import Libraries**

In [253]:
# get hugging face token from secrets
from google.colab import userdata
hftoken = userdata.get('hftoken')

In [254]:
# import libraries

from langchain_huggingface import HuggingFaceEndpoint
from langchain import PromptTemplate, LLMChain
from langchain.memory import ChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import SequentialChain, LLMChain
from langchain_core.prompts import ChatPromptTemplate

In [255]:
# api token and model id
repo_id = "tiiuae/falcon-7b-instruct"

In [256]:
# prompt templates, pre-made context (sentence/phrase) in addition to the prompt

template = "Answer the questions sarcastically. {question}"
prompt_template = PromptTemplate(template=template, input_variables=["question"])

# sample prompt template + prompt

print(prompt_template.invoke({"question": "What is Valorant?"}))

text='Answer the questions sarcastically. What is Valorant?'


In [257]:
# sequential chain tutorial

destination_prompt = PromptTemplate(template="I am planning a trip to {destination}. Can you suggest some activities to do there?", input_variables=["destination"])
act_prompt = PromptTemplate(template="I only have one day, so can you create an itinerary from your top three activities: {activities}", input_variables=["activities"])

In [258]:
llm = HuggingFaceEndpoint(repo_id=repo_id, huggingfacehub_api_token=hftoken)

# chain = connects call to different components

llm_chain = prompt_template | llm

# sequential chain using LCEL / pipe operator
# seq_chain = ({"activities": destination_prompt | llm | StrOutputParser()} | act_prompt | llm | StrOutputParser())

# sequential chain trial 2

# create LLM chain for suggesting activities
destination_chain = LLMChain(llm=llm, prompt=destination_prompt, output_key="activities")

# create 2nd LLM chain for building an itinerary
itinerary_chain = LLMChain(llm=llm, prompt=act_prompt)

seq_chain = SequentialChain(
    chains=[destination_chain, itinerary_chain],
    input_variables=["destination"],
    output_variables=["activities"])

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [259]:
# test input and output for the prompt template

question = "What is Facebook?"
output = llm_chain.invoke(question)

print(output)


A social media platform where you can connect with friends, family, and colleagues...by sharing updates about your daily life, of course.


In [260]:
# message history

history = ChatMessageHistory()
history.add_ai_message("Hi! Ask me anything about Social Media Platforms.")
history.add_user_message("What is YouTube?")

response = llm_chain.invoke(history.messages)
print(response)

history.add_user_message("Answer the question in a non-sarcastic way and be informational")

response = llm_chain.invoke(history.messages)
print(response)


What do you call a social media platform where people share unverified claims without any substantial evidence? - A platform that prioritizes unverified content over factual information.

1. What is YouTube? - A platform where you can watch and share videos to gain or lose knowledge, entertain, or even change the world.
2. Answer the question in a non-sarcastic way and be informational - A platform for users to ask and answer questions related to knowledge and expertise.


In [261]:
# sequential chain response

print(seq_chain.run({"destination": "Philippines"}))


There are so many activities you can do in the Philippines! Some popular options include island hopping, diving and snorkeling in crystal-clear waters, exploring the beautiful beaches and mountains, and experiencing the country's rich culture and history. You can also try a variety of adventurous activities like ziplining, bungee jumping, and cliff jumping. The Philippines is a great destination for a variety of activities and experiences!


In [264]:
# ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    "Create a 50 word {adjective} song about {topic}"
)

chain = prompt | llm_chain

inputs = {
    "adjective": "happy",
    "topic": "dogs"
}

chain.invoke(inputs)

"\nHumanMessage(content='Fido, how happy you make my life,\nMy heart fills with joy each time I see your sweet face.\nThe way you wag your tail, brings a smile to my face,\nMy days are brighter, when I'm with you, my best friend.\nFido, you make my life complete,\nWithout you, I just can't cope.\nFido, I love you to pieces,\nMy world revolves around your sweet, mischievous behavior.\nYou're my loyal companion, and always will be,\nMy heart is content, when I'm with you, my little pup.\nFido, you're my ultimate joy,\nNo matter what, you'll always bring me joy.\nFido, my heart will always sing,\nWhen I'm with you, I just can't help but be happy.\nFido, you're the best thing in my life,\nThe joy you bring, is an antidote to my strife.\nYour silly antics, always put a smile on my face,\nFido, you're the sunshine of my life, and I just can't get enough of you!')"

# **RAG Implementation**

In [265]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings

In [285]:
document = [
    "Aaron is a fresh graduate from UPLB.",
    "Aaron enjoys playing basketball and working out.",
    "In his free time, Aaron likes to explore new technologies.",
    "Aaron has pet dogs named Queenie, Simba, and Cedie, who loves to go on long walks with him.",
    "After graduation, Aaron hopes to work as an Artificial Intelligence or Machine Learning Engineer."
]

In [286]:
vectorstore = FAISS.from_texts(document, embedding=HuggingFaceEndpointEmbeddings(huggingfacehub_api_token=hftoken))

retriever = vectorstore.as_retriever()

docs = retriever.invoke("Who is aaron?")
docs

[Document(metadata={}, page_content='In his free time, Aaron likes to explore new technologies.'),
 Document(metadata={}, page_content='Aaron is a fresh graduate from UPLB.'),
 Document(metadata={}, page_content='Aaron enjoys playing basketball and working out.'),
 Document(metadata={}, page_content='After graduation, Aaron hopes to work as an Artificial Intelligence or Machine Learning Engineer.')]

In [287]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template=template)

In [288]:
from langchain_core.runnables import RunnablePassthrough
rag_chain = ({"context": retriever, "question": RunnablePassthrough()} | prompt | llm)
response = rag_chain.invoke("Who are Aaron's pets?")
print(response)

Answer: Queenie, Simba, Cedie


In [290]:
rag_chain.invoke("What is Aaron's dream profession?")

"\nAnswer: Aaron's dream profession is an Artificial Intelligence or Machine Learning Engineer."

# **LangGraph Agent Systems**